In [3]:
from tdw.controller import Controller
from tdw.tdw_utils import TDWUtils
from tdw.add_ons.third_person_camera import ThirdPersonCamera
from typing import Dict
from tdw.add_ons.collision_manager import CollisionManager
from tdw.add_ons.object_manager import ObjectManager

# Added for video
from tdw.add_ons.image_capture import ImageCapture
from tdw.backend.paths import EXAMPLE_CONTROLLER_OUTPUT_PATH
from os import chdir, system
from subprocess import call
import shutil

# Added for collisions
import random
import numpy as np

# From &tdw_physics
from typing import List, Dict
from tdw.librarian import ModelLibrarian
from typing import Dict
from helpers import ObjectInfo, get_random_avatar_position

In [11]:
lib = ModelLibrarian('models_core.json')
records = {record.name:i for i, record in enumerate(lib.records)}

In [ ]:
lib = ModelLibrarian('models_full.json')


In [12]:
lib.records[0].name

'034_vray'

In [7]:
print(records['rh10'])

2049


In [2]:
records = {record.name:record for record in lib.records}

NameError: name 'lib' is not defined

In [9]:
records['rh10'].name

'rh10'

: 